# データサイエンス応用 #1 課題解説

# プレーンテキストへの変換

## 課題1

「プレーンテキスト」とは、文字だけで構成され、レイアウト情報や装飾情報などを持たないデータのことです。
ここでは、これまで抽出してきたルビ等の補足情報等も存在していないテキストデータのことをプレーンテキストと呼ぶことにします。

「青空文庫」のデータでは、以下のようにルビと注釈の記号が定義されています。
    
    -------------------------------------------------------
    【テキスト中に現れる記号について】

    《》：ルビ
    （例）吾輩《わがはい》

    ｜：ルビの付く文字列の始まりを特定する記号
    （例）一番｜獰悪《どうあく》

    ［＃］：入力者注　主に外字の説明や、傍点の位置の指定
       （数字は、JIS X 0213の面区点番号またはUnicode、底本のページと行数）
    （例）※［＃「言＋墟のつくり」、第4水準2-88-74］

    〔〕：アクセント分解された欧文をかこむ
    （例）〔Quid aliud est mulier nisi amicitiae& inimica〕
    アクセント分解についての詳細は下記URLを参照してください
    http://www.aozora.gr.jp/accent_separation.html
    -------------------------------------------------------

`wagahaiwa_nekodearu.txt`から、上記のルビと注釈の部分を削除してプレーンテキストを出力`print(）`するプログラムを実装しなさい。

ただし、「※」は、外字（常用漢字以外等の出力できない文字）を示す記号であり、適切な文字を補うことは難しいので、そのまま残しても良いことにします。

    （例）
    処理前：主人はまだ※［＃「言＋墟のつくり」、第4水準2-88-74］《いつ》わられた事に気がつかない。
    処理後：主人はまだ※わられた事に気がつかない。
    
 ## 課題2
 
課題1に加えて、ルビと注釈の部分を削除したプレーンテキストの中に「猫」という文字が登場する回数をカウントし、その数を最後に出力`print(）`するコードを追加しなさい。

### ヒント
課題1と課題2を同時に実装したコードの出力例`wagahaiwa_nekodearu.output.txt`をMoodleコースに掲載します。
これと同様な出力ができればOKです。参考にしてください。

# 回答例

課題1と課題2を合わせた回答例を示します。

---
    import re

    path = "wagahaiwa_nekodearu.txt"

    pattern = re.compile(r"《[^《》]*》|［[^［］]*］|｜")
    pattern2 = re.compile( r"〔[^〔〕]*〕")

    with open(path, encoding="utf-8") as f:
            lines = f.readlines()

    neko_count = 0 #初期化
    for l in lines:
            l_strip = l.strip()

            result = pattern.sub("",  l_strip)
            final_result = pattern2.sub("", result)
            print(final_result)

            neko_match = re.findall("猫", final_result)
            neko_count += len(neko_match)

    print("\n「猫」の登場回数は、" + str(neko_count) + "である。")

この例では、正規表現のパターンは、
    
    pattern = re.compile(r"《[^《》]*》|［[^［］]*］|｜")

としており、`｜`（全角の｜）を追加しています。

これら記号を実際に削除しているのが、

    result = pattern.sub("",  l_strip)

です。`pattern.sub()`で置換していますが、置換先が `""` で空になっています。

また、〔〕が以下の箇所に含まれていましたので、これも削除してみましょう。

    何だって？　〔Quid aliud est mulier nisi amicitiae&［＃「〔amicitiae&〕」は底本では「amiticiae」］ inimica〕……こりゃ君｜羅甸語《ラテンご》じゃないか」

ここでは〔〕の中で［］も入れ子になっています。 そのため、まずは上記の正規表現と`pattern.sub()`を適用して、まずは、

    何だって？　〔Quid aliud est mulier nisi amicitiae& inimica〕……こりゃ君羅甸語じゃないか」

の状態（`［］`を削除）にしたあとに、

    pattern2 = re.compile( r"〔[^〔〕]*〕")
    final_result = pattern2.sub("", result)

をさらに適用するようにしてみました（方法はこれ以外にもいろいろあります）。

「猫」の回数をカウントしているのは、

    neko_match = re.findall("猫", final_result)
    neko_count += len(neko_match)
    
です。`re.findall()`は、正規表現パターン（この場合は、"猫"のみ)にマッチした要素をすべて返すため、その数を数える（`len(neko_match)`）ことで行ごとの登場回数を得ることができます。

それを`neko_count`という変数に合計（`+=`）しています。

なお、`neko_count`は、初期化（`neko_count = 0`）するようにしましょう。変数の初期化を忘れると、想定外のバグになることがあるので気を付けてください。

## raw文字列（ロー文字列）とバックスラッシュ（＼）

ところで、これまで正規表現のパターンを書くときに、`""`で定義した文字列の先頭に`r`をつけているのが気になった人もいると思います。 Pythonでは、これをraw文字列（raw string） といい、バックスラッシュ（`\`）によるエスケープシーケンスを特別扱いしない、 つまり、`print()`等の中で使うことができる`\n`（改行）などの特殊文字を解釈しない文字列となります。

例えば、

    print("one\ntwo\nthree")

は、

    one
    two
    three

となります。`\n` は、改行の特殊文字として扱われます。

一方で、raw文字列の場合は、特殊文字は解釈しないため、

    print(r"one\ntwo\nthree")

は、

    one\ntwo\nthree

と出力されます。

正規表現を使う場合、`\n`（改行）などの特殊文字以外に、正規表現の特殊シーケンス（`\d`や`\s`など）があり、 バックスラッシュ（`\`） を多用するため、パターンの定義の際にはraw文字列を使用することになっています。

なお、raw文字列を使わない場合、バックスラッシュ（`\`）自体を文字列として表記したいときには、`\\`のようにバックスラッシュを2つ重ねます。

また、ここまでバックスラッシュ（`\`）と書いてきましたが、日本語の環境の多くでは円記号（`¥`）が表示されていると思います。 ただし、バックスラッシュの本当は、`＼`の半角です。 日本のコンピュータや日本語フォント・OS環境ではバックスラッシュを円記号として表示するものが多くなっており、 その場合は、円記号をバックスラッシュと同じ処理とすることになっています。

# 正規表現を使わない文字列操作

正規表現では、パターンを定義することで、文字列の中から、パターンにマッチする部分を置換したり、削除したりできることを解説しました。
ただし、Pythonの文字列型のオブジェクトでは、固定された定型部分であれば、もっと簡単に文字列を操作することができます。

## カウント

例えば、以下の例では、`count()`を利用することで、テキスト中に登場する「猫」という文字の回数をカウントしています。

---
    import re

    path = "wagahaiwa_nekodearu.txt"

    pattern = re.compile(r"《[^《》]*》|［[^［］]*］|｜")
    pattern2 = re.compile( r"〔[^〔〕]*〕")

    with open(path, encoding="utf-8") as f:
            lines = f.readlines()

    count = 0
    neko_count = 0
    for l in lines:
            count = count + 1
            l_strip = l.strip()

            result = pattern.sub("",  l_strip)
            final_result = pattern2.sub("", result)
            print(final_result)

            neko_count += final_result.count("猫")

    print("\n「猫」の登場回数は、" + str(neko_count) + "である。")


## 置換と削除

`replace()`を使うことで、簡単に文字列の一部を他の文字列に置き換えることができます。
なお、`replace()`は、文字の削除にも使えます。 2番目の引数に空の文字列（`""`） を与えることによって、文字列を削除できます。

----
    input_text = "私は元気です。"

    output_text = input_text.replace("元気", "たぬき")
    print("input: " + input_text)
    print("output: " + output_text)

    print()

    output_text = input_text.replace("私は", "")
    print("input: " + input_text)
    print("output: " + output_text)

文字列を整数型に変換するのならば、`int()`を使うことができます。

---
    text = "5000000"
    num = int(text)
    print(num)

---

ただし、文字列の中にカンマのような記号が入っていると、`int()`で変換する段階でエラーになります。
この場合は、`replace()`を使って、カンマを削除してからint()で変換するようにしましょう。

---
    price = "5,000,000"

    #カンマを削除してint()で変換
    num = int(price.replace(",", ""))

    # 消費税の計算
    price_tax = num * 1.1

    print("税込み" + str(price_tax) + "円です。")

## 一致判定

ある文字列にある文字列を含むか否かの判定には、簡単なものは、正規表現を使用しないでも、`in`, `startswith()`, `endswith()`を利用することができます。特に`in`は手軽ですので多くの場合で利用されます。

    if "元気" in input_text:

で、`input_text`の中に「`元気`」という文字列が含まれるか判定（部分一致）することができます。

先頭から含まれるか（前方一致）には`startswith()`、末尾に含まれるか（後方一致）には`endswith()`を利用できます。
なお、完全一致には、`==`を利用してください。

    if input_text == "元気":

のようになります。

---
    input_text = "私は元気です。"

    print("input: " + input_text)

    # 文字列の部分一致
    if "元気" in input_text:
        print("元気は含まれます。")
    else:
        print("元気は含まれていません。")

    # 前方一致（文字列の先頭が指定した文字列から始まっているか）
    if input_text.startswith("私は"):
        print("「私は」からはじまる文字列")
    else:
        print("「私は」からはじまっていない")

    # 後方一致（文字列の末尾が指定した文字列で終わっているか）
    # は endswith() を使うことができる

# （重要）ファイルへの保存

最後に、前述の回答例のプログラムが出力する結果（プレーンテキスト）をファイルに保存する方法を示します。
ここで出力する`plain.txt`は、後から利用しますので、必ず、このコードを実行して`plain.txt`を保存するようにしておいてください。

---
    import re

    input_file = "wagahaiwa_nekodearu.txt"
    output_file = "plain.txt"

    pattern = re.compile(r"《[^《》]*》|［[^［］]*］|｜")
    pattern2 = re.compile( r"〔[^〔〕]*〕")

    with open(input_file, encoding="utf-8") as f:
            lines = f.readlines()

    texts = []; # 初期化
    for l in lines:
            l_strip = l.strip()

            result = pattern.sub("",  l_strip)
            final_result = pattern2.sub("", result)
            #print(final_result)
            texts.append(final_result)

    with open(output_file, mode='w', encoding="utf_8") as f:
        f.write('\n'.join(texts))
        
    print("done.")

このプログラムでは、入力ファイル（`wagahaiwa_nekodearu.txt`）と出力ファイル（`plain.txt`)のファイル名が、`input_file`と`output_file`という変数に保存されます。

前述の回答例では、
    
    print(final_result)
    
として、出力を`print()`していましたが、画面に出力するのではなく、`texts`という名前のリストに追加（`append`）するようにしています。

    texts.append(final_result)
    
によって、`final_result`の中身を`texts`に追加しています。`texts`の中に、行ごとにどんどん結果が溜まるわけですね。なお、`texts`は、使用する前に
     
    texts = []; # 初期化
    
として初期化（中身をからっぽに）しています。

次に、

    with open(output_file, mode='w', encoding="utf_8") as f:
    
によって、`output_file`をファイルの書き込みモードwrite mode（`mode='w'`）で`open()`しています。

`open()`できたら、`f.write()`（文字列を書き込み）や`f.writelines()`（リストを書き込み）で書き込めばよいことになります。

つまり、

    f.writelines(texts)
    
とすることで、`texts`の中身を全部書き出すことができます。ここで`texts`は行ごとに分割されていますが、各行末の`\n`が無い状態になっているため、そのままだと出力結果には改行（`\n`）がすべて存在しない状態になります（すべての行が一つにつながった状態になる）。

それを防ぐために、
    
    `\n`.join(texts)
    
によって、行ごとに分割された`texts`を改行（`\n`）で結合（join）して、一つの改行（ `\n` ）込みの文字列に繋げてから、`write()`するようにしています。

なお、保存された`plain.txt`は、Jupyter Notebookのトップ（ファイルの一覧）から、ファイル名をクリックして、中身を確認することができます。